Data download from https://www.notion.so/977d5e5be0434bf996704ec361ad621d?v=fe54f89ca9e04ac799af42b39e1efc4b -> export -> as CSV

In [6]:
import json
import pandas as pd
import numpy as np

In [7]:
cmap = json.load(open('country-mapping.json'))

In [8]:
df = (
    pd
    .read_csv('containment.csv')
    .dropna(subset=["Country"])
    .assign(
    iso_alpha_3=lambda x: x["Country"].map(
        lambda y: cmap['US'] if y.startswith('US:') else cmap[y]),
    sort_date_start=lambda df: df["Date Start"].map(pd.to_datetime),
    date_start=lambda df: df["sort_date_start"].map(lambda x: x.isoformat()),
    date_end=lambda df: df["Date end intended"].map(lambda x: pd.to_datetime(x).isoformat()).map(lambda x: None if x == "NaT" else x),
    keywords=lambda x: x["Keywords"].dropna().map(lambda x: [y.strip() for y in x.split(',')]),
    )
    .sort_values('sort_date_start', ascending=True)
    .rename(columns={
        'Description of measure implemented': "description",
        "Source": "source",
        "Country": "country",
    }
    )
    .filter(items=["iso_alpha_3", "date_start", "date_end", "country", "description", "keywords", "source"])
    .dropna(subset=["description"])
    .set_index('iso_alpha_3')  # not unique!
)

In [9]:
final_d = {}
for iso in df.index.unique():
    final_d[iso] = [rec.replace({np.nan: None}).to_dict() for _, rec in df.loc[[iso]].iterrows()]

In [10]:
with open('data-containments.json', "w") as ofile:
    json.dump(final_d, ofile)

## Structure

```
{
   "<ISO-ALPHA3-CODE>": [
       {"date_start": str, "date_end": str, "country": str, "keywords": List[str], source: str}
   ]
}
```

## "Test"

In [11]:
json.load(open('data-containments.json'))["VAT"]

[{'date_start': '2020-03-10T00:00:00',
  'date_end': None,
  'country': 'Vatican City',
  'description': 'Italy lockdown measures are also applied to the Vatican City.',
  'keywords': ['blanket isolation - no symptoms',
   'religious activity limitations',
   'social distancing'],
  'source': 'https://www.theguardian.com/world/video/2020/mar/10/st-peters-square-before-and-after-italys-nationwide-lockdown-video'}]

## Upload

In [12]:
# this won't work for you if you don't have permissions
! gsutil -h "Cache-Control:public, max-age=10" cp -a public-read data-containments.json gs://static-covid/static/data-main-containments.json

Copying file://data-containments.json [Content-Type=application/json]...
- [1 files][567.1 KiB/567.1 KiB]                                                
Operation completed over 1 objects/567.1 KiB.                                    
